# Computer vision

## Image classification

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nlphuji/flickr30k")
image = dataset['test'][134]["image"] 

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image) 

In [ ]:
from transformers import pipeline

pipe = pipeline("image-classification", "google/mobilenet_v2_1.0_224")
pred = pipe(image) 
print("Predicted class:", pred[0]['label'])

## Object detection

In [ ]:
pipe = pipeline("object-detection", "facebook/detr-resnet-50", revision="no_timm")
outputs = pipe(image, threshold=0.95)

for obj in outputs:
  box = obj['box'] 
  print(f"Detected {obj['label']} with confidence {obj['score']:.2f} at ({box['xmin']}, {box['ymin']}) to ({box['xmax']}, {box['ymax']})")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

ax = plt.gca()
colors = ['r', 'g', 'b', 'y', 'm', 'c', 'k']
plt.imshow(image)  
for n, obj in enumerate(outputs):
  box = obj['box']   
  rect = patches.Rectangle(
    (box['xmin'], box['ymin']),
    box['xmax']-box['xmin'],
    box['ymax']-box['ymin'],
    linewidth=1,
    edgecolor=colors[n],
    facecolor='none')  
  ax.add_patch(rect)  
plt.show() 

## Image background removal

In [ ]:
pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)
outputs = pipe(image)

plt.imshow(outputs)
plt.show()